# TAIYŌAI INC.

## Data Engineering Trial Task

### Devansh Kasaudhan


In [3]:
#@title Importing Modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

In [2]:
#@title Class Deffination
class Person:
    def __init__(self,a ):
        self.a = a
    def abc(b):
        csv_file = []
        base_url = b.a
        for page in tqdm.tqdm(range(1, 125)):
            r = requests.get(base_url + str(page) + "#dashboard_notices.html")
            c = r.content
            soup = BeautifulSoup(c, 'html.parser')
            all_data = soup.find_all("div", {"class": "search-result"})
            #print(all_data)
            for item in all_data:
                d = {}
                d["Tender"] = item.find("a", {"class": ["govuk-link", "search-result-rwh", "break-word"]}).text
                d["Company"] = item.find("div", {"class": ["search-result-sub-header", "wrap-test"]}).text
                d["Procurement"] = item.find_all("div", {"class": "search-result-entry"})[0].text.replace("Procurement stage", " ")
                d["Notice"] = item.find_all("div", {"class": "search-result-entry"})[1].text.replace("Notice status"," ")
                d["Location"] = item.find_all("div", {"class": "search-result-entry"})[3].text.replace("Contract location", " ")
                try:
                    d["Closing"] = item.find_all("div", {"class": "search-result-entry"})[2].text.replace("Closing"," ")
                except:
                    d["Closing"] = "None"
                try:
                    d["Value"] = item.find_all("div", {"class": "search-result-entry"})[4].text.replace("Contract value", " ")
                except:
                    d["Value"] = "None"
                try:
                    d["Date"] = item.find_all("div", {"class": "search-result-entry"})[5].text.replace("Publication date", " ")
                except:
                    d["Date"] = "None"
                csv_file.append(d)
        df=pd.DataFrame(csv_file)
        df.to_csv("Scraper.csv",index=False)
p1 = Person("https://www.contractsfinder.service.gov.uk/Search/Results?&page=")
p1.abc()

100%|██████████| 124/124 [08:05<00:00,  3.92s/it]


In [6]:
#@title Loading Dataset
df= pd.read_csv("Scraper.csv")

In [7]:
df.head()

,Tender,Company,Procurement,Notice,Location,Closing,Value,Date
0,UCLH-4376 - Catering Provision @ 250 Euston Road,University College London Hospitals NHS Founda...,Opportunity,Open,United Kingdom,"21 June 2024, 12pm","£225,000",29 April 2024
1,CSI/PH BNSSG Integrated Sexual Health Service,Bristol City Council,Opportunity,Open,England,"24 June 2024, 12pm","£64,625,880","29 April 2024, last edited 29 April 2024"
2,GSP1314 Supply Leather Hides for the House of ...,House of Commons,Opportunity,Open,SW1A 0AA,"29 May 2024, 12pm","£150,000",29 April 2024
3,Pest Control Services,Mhs Homes Group,Opportunity,Open,South East,"31 May 2024, 12pm","£180,000 to £300,000",29 April 2024
4,Wolverley CE Secondary School Re-roofing Phase 2,Wolverley CE Secondary School,Opportunity,Open,West Midlands,"24 May 2024, 12pm","£350,000",29 April 2024


In [15]:
df_ner = df[["Tender"]]

In [16]:
import spacy
import warnings
warnings.filterwarnings('ignore')

In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [19]:
#@title Named Entity recognition of tender column
df_ner["Extracted Entities"] = ""

for i, row in df.iterrows():
    project_name = row["Tender"]
    entities = extract_entities(project_name)
    df_ner.at[i, "Extracted Entities"] = entities

In [21]:
df_ner.head()

,Tender,Extracted Entities
0,UCLH-4376 - Catering Provision @ 250 Euston Road,"[(UCLH-4376, GPE), (250, CARDINAL)]"
1,CSI/PH BNSSG Integrated Sexual Health Service,[(CSI/PH BNSSG Integrated Sexual Health Servic...
2,GSP1314 Supply Leather Hides for the House of ...,"[(the House of Commons Benches, ORG)]"
3,Pest Control Services,"[(Pest Control Services, ORG)]"
4,Wolverley CE Secondary School Re-roofing Phase 2,"[(Phase 2, PRODUCT)]"
